### 라이브러리 로드

In [ ]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import cv2

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

## Simple EDA

In [ ]:
axes = []
fig = plt.figure(figsize=(20,6))
for i in range (13):
    axes.append(fig.add_subplot(2,7,i+1))
    subplot_title = str(i)
    axes[-1].set_title(subplot_title)
    img = Image.open('../input/pceo-mnist/train/'+str(i)+'/'+'G_0_0_0_0_0.png')
    plt.imshow(img)
fig.tight_layout()
plt.show()

### 데이터 전처리

In [ ]:
sample_submission = pd.read_csv('../input/pceo-mnist/sample_submission.csv')
sample_submission.head()

In [ ]:
train_dir = '../input/pceo-mnist/train'
test_dir = '../input/pceo-mnist/test'
n_classes = 13
class_label = [i for i in range (n_classes)]
target_size = (28,28)
target_dim = (28,28,3)
val_size = 0.2
batch_size = 64

In [ ]:
data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True, 
                                    validation_split=val_size,
                                    zoom_range=0.1,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1)
test_data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True)

train_generator = data_augmentor.flow_from_directory(train_dir, target_size=target_size, batch_size=batch_size, shuffle=True, subset="training")
val_generator = data_augmentor.flow_from_directory(train_dir, target_size=target_size, batch_size=batch_size, subset="validation")
test_generator = test_data_augmentor.flow_from_directory(test_dir, target_size=target_size, batch_size=batch_size, shuffle=False)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
 
early_stopping = EarlyStopping(
     min_delta = 0.001,
     patience = 4,
     restore_best_weights=True
)

### 모델링 및 훈련

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), activation='relu', input_shape=target_dim))
model.add(MaxPool2D(pool_size=2,strides=1,padding="same"))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
with tf.device("/device:CPU:0"):
    model.fit_generator(train_generator, epochs=20, validation_data = val_generator, callbacks = [early_stopping])

### Predict 및 Submission

submission을 위한 부분을 변경하지 말아주세요. 아래 코드로 실행하여 submission 파일을 만들어야 올바르게 채점할 수 있습니다. 

In [ ]:
test_generator.reset()
predict = model.predict_generator(test_generator)
result = np.argmax(predict, axis=1)

In [ ]:
submission = sample_submission.copy()
for i in range(len(submission)):
    submission.iloc[i,0] = str(submission.iloc[i,0])
submission = submission.sort_values('file_index')

In [ ]:
class_map = train_generator.class_indices

In [ ]:
submission = submission.reset_index()
for i in range(len(submission)):
    submission.iloc[i,-1] = list(class_map.keys())[list(class_map.values()).index(result[i])]
submission = submission.drop('index', axis=1)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)